# Llama 3.2 한글 Test

Test Model

In [ ]:
%pip install transformers==4.46.3
%pip install datasets==3.2.0
%pip install accelerate==1.2.1
%pip install peft==0.14.0
%pip install trl==0.12.2
%pip install bitsandbytes==0.45.0

In [5]:
# The model that you want to train from the Hugging Face hub
#base_model = "Bllossom/llama-3.2-Korean-Bllossom-3B"
base_model = "rssaem/Llama3_2_ModelBTS"

# Base Model

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer
import torch

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import torch
res = torch.cuda.is_available()
print(res)

True


In [6]:

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Set pad_token_id to avoid receiving warning messages.

In [7]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [8]:
def generate_response(messages, model):

    input_ids = tokenizer.apply_chat_template(
         messages, add_generation_prompt=True,
         return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    decoded_output = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    return decoded_output

instruction = "BTS 음악의 컨셉은 무엇입니까?"
messages = [
    {"role": "user", "content": f"{instruction}"},
]

generate_response(messages, model)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"BTS는 다양한 음악 컨셉을 통해 팬들과 상호작용하고 있습니다. 그 중 몇 가지 주요한 컨셉은 다음과 같습니다:\n\n1. **'불타오르는 사랑' 컨셉**: BTS는 '불타오르는 사랑'을 주제로 한 음악을 통해 사랑과 감정에 대한 깊은 이해를 보여줍니다. 이 컨셉은 팬들과의 깊은 결속을 강화하는 데 큰 역할을 합니다.\n\n2. **'자아 발견' 컨셉**: BTS는 자신들의 삶과 생각을 주제로 한 음악을 통해 팬들과의 상호작용을 강화하고자 합니다. 이 컨셉은 팬들이 자신을 더 잘 이해하고, 자신감을 얻을 수 있는 기회를 제공합니다.\n\n3. **'사회적 문제' 컨셉**: BTS는 사회적 문제와 관련된 음악을 통해 팬들과의 대화와 상호작용을 강화하고자 합니다. 이 컨셉은 팬들이 더 많은 것을 알고, 사회적 문제에 대한 이해를 높이는 데 기여합니다.\n\n4. **'고독과 분명함' 컨셉**: BTS는 고독과 분명함을 주제로 한 음악을 통해 팬들과의 깊은 감정 공유를 통해 상호작용을 강화하고자 합니다. 이 컨셉은 팬들이 자신을 더 잘 이해하고, 자신감을 얻을 수 있는 기회를 제공합니다.\n\n이 외에도 BTS는 다양한 컨셉을 통해 팬들과의 상호작용을 강화하고 있습니다. 팬들과의 깊은 결속과 상호작용을 통해 음악을 더 깊고 의미 있는 방식으로 즐길 수 있게 합니다."

Create the text generation pipeline with model and tokenizer.

In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Run the pipeline with the custom prompt

In [10]:

messages = [{"role": "user", "content": "BTS 음악의 컨셉은 무엇입니까?"}]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=terminators, do_sample=True)

print(outputs[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Dec 2024

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

BTS 음악의 컨셉은 무엇입니까?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

BTS는 다양한 컨셉을 통해 음악을 전개하고 있습니다. 그들의 음악은 주로 정서, 사회적 문제, 개인의 성장, 사랑, 그리고 삶의 의미를 다루고 있습니다. 예를 들어, 'DNA', 'Mic Drop', 'Boy With Luv' 등의 곡은 에너지와 감정의 강도 높은 컨셉을, 'Spring Day', 'Euphoria', 'Butter' 등의 곡은 정서적 감정을 강조하며, 'Life Goes On', 'Dynamite' 등의 곡은 긍정적이고 희망적인 메시지를 전달하고 있습니다.


In [11]:
messages2 = [{"role": "user", "content": "BTS 음악의 컨셉을 3개만 적어주세요."}]

prompt2 = tokenizer.apply_chat_template(
    messages2, tokenize=False, add_generation_prompt=True
)

outputs2 = pipe(prompt2, max_new_tokens=1024, eos_token_id=terminators, do_sample=True)

print(outputs2[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Dec 2024

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

BTS 음악의 컨셉을 3개만 적어주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

BTS의 음악은 다양한 컨셉을 가지고 있습니다. 주요 컨셉은 다음과 같습니다:

1. **자아 발견과 성장**: BTS의 음악은 주로 개인의 자아를 발견하고 성장하는 과정에 초점을 맞춥니다. '다이너마이트', '피 땀과 눈물', '다시 born' 등이 이 컨셉을 대표합니다.

2. **사회적 문제와 사회적 변화를 주제로**: BTS는 다양한 사회적 문제와 사회적 변화를 주제로 음악을 제작하고 있습니다. '마이너리티', '시그널', '아이돌' 등이 이 컨셉을 보여줍니다.

3. **사랑과 relationships**: 사랑과 관계를 주제로 하는 음악도 많이 있습니다. '보라색 바다', '세븐', '마이너리티' 등이 이 컨셉을 대표합니다.
